In [3]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus

from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection
import ccdproc as ccdp

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
from convenience_functions import show_image
from astropy.timeseries import BinnedTimeSeries, TimeSeries
from astropy.table import Column

from astropy import units as u
from astropy.coordinates import SkyCoord
from photutils.aperture import SkyCircularAperture


# suppress warnings is not recommended :I
import warnings
warnings.filterwarnings("ignore")

In [8]:
cc_com_reduced = './data/cc_com/reduced/'
tz_boo_reduced = './data/tz_boo/reduced/'

def get_aperture_fluxes(images_dir, image_dim=(3056, 3056), r=50.0):
    images = ImageFileCollection(images_dir)
    images = images.files_filtered(include_path=True)
    
    mask = np.ones(image_dim, dtype=bool)
    mask[1400:1600, 1400:1600] = False
    
    APT_fluxes = []
    ANU_fluxes = []
    for i, path in np.ndenumerate(images):
        # print(path)
        image = CCDData.read(path)

        position = np.transpose((
            image.header['X-CENTROID'],
            image.header['Y-CENTROID']
        ))
        aperture = CircularAperture(position, r=r)
        annul = CircularAnnulus(position, r_in=r, r_out=1.5*r)
        phot_circ = aperture_photometry(image, aperture)[0]['aperture_sum']
        phot_anul = aperture_photometry(image, annul)[0]['aperture_sum']
        
        image.header['APT-FLUX'] = phot_circ.value
        image.header['ANU-FLUX'] = phot_anul.value
        
        APT_fluxes.append(phot_circ.value)
        ANU_fluxes.append(phot_anul.value)
        
    return APT_fluxes, ANU_fluxes

In [9]:
tzboo_apt, tzboo_anu = get_aperture_fluxes(tz_boo_reduced)
print(tzboo_apt)
print(tzboo_anu)

664613.8785686388 adu
612967.0976914911 adu
583117.2724831183 adu
544382.3021524142 adu
[696743.7145410595, 672665.0032227268, 657920.2438691185, 602233.4861799859]
[664613.8785686388, 612967.0976914911, 583117.2724831183, 544382.3021524142]


In [10]:
cccom_apt, cccom_anu = get_aperture_fluxes(cc_com_reduced)
# cccom_fluxes.write('./results/cccom_DAOresults.csv')
print(cccom_apt)
print(cccom_anu)

548667.5388424691 adu
545002.6669727325 adu
556507.338906915 adu
[450925.6694071289, 446639.445057526, 455304.25499404693]
[548667.5388424691, 545002.6669727325, 556507.338906915]


In [16]:
tzboo_DAO = BinnedTimeSeries.read(
    './results/tzboo_DAOresults.csv',
    time_bin_start_column='time_bin_start',
    time_bin_size_column='time_bin_size',
    time_bin_size_unit=u.s
)
apflux_col = Column(name='AP flux', data=tzboo_apt)
anflux_col = Column(name='Anu flux', data=tzboo_anu)
tzboo_DAO.add_column(apflux_col)
tzboo_DAO.add_column(anflux_col)
tzboo_DAO.write('./results/tzboo_results.csv', overwrite=True)

In [17]:
tzboo_DAO

time_bin_start,time_bin_size,xcentroid,ycentroid,DAO flux,AP flux,Anu flux
,s,,,,,
Time,float64,float64,float64,float64,float64,float64
2023-05-05T03:37:43.990,15.0,1505.658545068577,1451.9598233141166,45.283864574166145,696743.7145410595,664613.8785686388
2023-05-05T04:06:02.330,15.0,1486.5070405122656,1459.4518074772132,70.99202021518356,672665.0032227268,612967.0976914911
2023-05-05T04:33:47.270,15.0,1529.8081988179474,1527.4394505568735,128.77656611821413,657920.2438691185,583117.2724831183
2023-05-05T05:33:33.170,15.0,1524.2979148448526,1521.6222535867441,102.93023116168068,602233.4861799859,544382.3021524142


In [13]:
cccom_DAO = BinnedTimeSeries.read(
    './results/cccom_DAOresults.csv',
    time_bin_start_column='time_bin_start',
    time_bin_size_column='time_bin_size',
    time_bin_size_unit=u.s
)
apflux_col = Column(name='AP flux', data=cccom_apt)
anflux_col = Column(name='Anu flux', data=cccom_anu)
cccom_DAO.add_column(apflux_col)
cccom_DAO.add_column(anflux_col)
cccom_DAO.write('./results/cccom_results.csv', overwrite=True)

In [14]:
cccom_DAO

time_bin_start,time_bin_size,xcentroid,ycentroid,DAO flux,AP flux,Anu flux
,s,,,,,
Time,float64,float64,float64,float64,float64,float64
2023-05-05T03:47:49.549,15.0,1553.040442438791,1437.5034251892966,3.319052059965849,450925.6694071289,548667.5388424691
2023-05-05T04:15:01.570,15.0,1551.4548739944155,1438.76161766664,4.407717909508814,446639.445057526,545002.6669727325
2023-05-05T04:58:12.580,15.0,1550.1356753563516,1438.672920101272,5.94613799804669,455304.25499404693,556507.338906915


In [2]:
from photutils.psf import IterativelySubtractedPSFPhotometry
my_photometry = IterativelySubtractedPSFPhotometry(
    finder=my_finder, group_maker=my_group_maker,
    bkg_estimator=my_bkg_estimator, psf_model=my_psf_model,
    fitter=my_fitter, niters=3, fitshape=(7, 7))
# get photometry results
photometry_results = my_photometry(image=my_image)
# get residual image
residual_image = my_photometry.get_residual_image()

NameError: name 'my_finder' is not defined